### 构建息肉数据集

In [1]:
import torch
from torchvision import datasets, transforms
import torch
import torch.distributed as dist
import torch.nn as nn
from trainer.trainer import Trainer
from trainer.ssod_trainer import SSODTrainer
from configs.defaults import get_cfg
from utils.general import increment_path, check_git_status, check_requirements, \
    print_args, set_logging
from pathlib import Path
import logging
import os
import argparse
from utils.callbacks import Callbacks
from utils.torch_utils import select_device
import sys
from datetime import timedelta
import val
# 定义目标检测模型
from models.detector.yolo import Model
import json
import os

import numpy as np

import matplotlib.pyplot as plt
import random
import pathlib
import cv2

In [9]:
matrix=np.asmatrix([[          0,0,0,0,0,0,0,0,0,0,3],
 [          2,1,1,0,0,0,0,0,0,0,8],
 [          4,3,0,1,0,0,0,0,0,0,8],
 [          0,0,0,0,0,0,0,0,0,0,5],
 [          0,3,1,1,2,0,0,0,0,0 ,  13],
 [          0,0,0,0,0,0,0,0,0,0,0],
 [          0,0,0,0,0,0,0,0,0,0,0],
 [          0,0,0,0,0,0,0,0,0,0,0],
 [          0,0,0,0,0,0,0,0,0,0,0],
 [          0,0,0,0,0,0,0,0,0,0,0],
 [          2,6,1,0,0,0,0,0,0,0,0]])

In [13]:
print(sum(sum(sum(matrix))))

[[ 8 13  3  2  2  0  0  0  0  0 37]]


In [8]:
device = torch.device('cuda:0')

def load_model():
    ckpt = torch.load(r'E:\medical\depth\runs\exp15_polyp_yolov5\best.pt', map_location=device)  # load checkpoint
    model = ckpt['model']
    return model

In [27]:
class FCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # 定义全连接层
        self.fc1 = torch.nn.Linear(in_features=25200 * 6, out_features=1024, dtype=torch.half)

        self.fc2 = torch.nn.Linear(in_features=1024, out_features=1)

    def forward(self, x):
        # 将图像展平
        x = x.view(-1, 25200 * 6)

        # 全连接层
        x = self.fc1(x)        
        x = torch.nn.ReLU()(x)
        x = self.fc2(x)
        x = torch.nn.Sigmoid()(x)

        return x


In [10]:
# 微调分类模型
batch_size = 32
train_dataset = datasets.ImageFolder(root=r"E:\medical\depth\dataset\classify", transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((640, 640)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
model=load_model()

In [28]:
# 定义分类模型
classifier = FCNN().to(device)
classifier.cuda().half()
# 微调分类模型

optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
# 定义损失函数
criterion = torch.nn.BCEWithLogitsLoss().to(device)
input_shape = None
times=0
for epoch in range(1):
    for images, labels in train_loader:
        times+=1
        if times>2:
            break
        images = images.cuda().half()
        
        # 前向传播
        outputs = model(images)
        classify_input = outputs[0]

        if input_shape is None:
            input_shape = classify_input.shape
            print("yolo output shape:", input_shape, ",classify_input:", classify_input.shape)
        # 梯度清零
        optimizer.zero_grad()
        logits = classifier(classify_input)
        # 计算损失
        _labels = labels.view(-1, 1).to(device).cuda().half()
        print("classify output shape:", logits.shape, ", values:", logits)
        # print("labels shape:", _labels.shape, ", values:", _labels)

        loss = criterion(logits, _labels)
        print("loss:", loss.item())

        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

yolo output shape: torch.Size([32, 25200, 6]) ,classify_input: torch.Size([32, 25200, 6])
classify output shape: torch.Size([32, 1]) , values: tensor([[1.01328e-06],
        [3.87430e-04],
        [4.35114e-06],
        [2.98023e-06],
        [4.11272e-06],
        [1.43051e-06],
        [9.53674e-07],
        [5.78165e-06],
        [4.35114e-06],
        [2.95639e-05],
        [2.50340e-06],
        [5.72205e-06],
        [5.96046e-08],
        [2.92063e-06],
        [4.76837e-07],
        [4.94719e-06],
        [6.67572e-06],
        [1.78814e-06],
        [1.29342e-04],
        [2.38419e-06],
        [1.19209e-07],
        [7.39098e-06],
        [9.59635e-06],
        [1.78814e-07],
        [1.89543e-05],
        [4.17233e-06],
        [1.39475e-05],
        [6.25849e-06],
        [3.15905e-06],
        [8.94070e-07],
        [1.19209e-07],
        [5.18560e-06]], device='cuda:0', dtype=torch.float16, grad_fn=<SigmoidBackward>)
loss: 0.693359375
classify output shape: torch.Size([32